In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import matplotlib.ticker as ticker
import numpy as np
import datetime

In [2]:
# imports
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
from sklearn import set_config
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer

In [3]:
df= pd.read_csv('data/UPDATE_full_data_clean_inclCluster.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Store ID,Item ID,Store Category,Store Segment,Store Region,City,Item Last Supplying Date,Item Price,Date,Declared Supply,Meals Saved,Total Supply,Pickup Length,Supply Length,Region Language,Region Type,Cluster
0,0,0,11344,11438,Key Account,Bakery,Genève,Genève,2022-03-01,4.5,2019-01-01,20,2,20,0 days 00:50:00,1545,french,urban,3
1,1,1,11729,11822,Indie - Outbound,Buffet,Bern,Bern,2019-06-27,6.9,2019-01-01,0,9,16,0 days 04:00:00,237,german,urban,0
2,2,2,11729,11822,Indie - Outbound,Buffet,Bern,Bern,2019-06-27,6.9,2019-01-02,0,28,33,0 days 04:00:00,237,german,urban,0
3,3,3,11729,11822,Indie - Outbound,Buffet,Bern,Bern,2019-06-27,6.9,2019-01-27,10,11,17,0 days 04:00:00,237,german,urban,0
4,4,4,11729,11822,Indie - Outbound,Buffet,Bern,Bern,2019-06-27,6.9,2019-01-31,10,13,17,0 days 04:00:00,237,german,urban,0


In [4]:
df.drop(columns=['Unnamed: 0','Unnamed: 0.1'],inplace=True)
df.head()

,Store ID,Item ID,Store Category,Store Segment,Store Region,City,Item Last Supplying Date,Item Price,Date,Declared Supply,Meals Saved,Total Supply,Pickup Length,Supply Length,Region Language,Region Type,Cluster
0,11344,11438,Key Account,Bakery,Genève,Genève,2022-03-01,4.5,2019-01-01,20,2,20,0 days 00:50:00,1545,french,urban,3
1,11729,11822,Indie - Outbound,Buffet,Bern,Bern,2019-06-27,6.9,2019-01-01,0,9,16,0 days 04:00:00,237,german,urban,0
2,11729,11822,Indie - Outbound,Buffet,Bern,Bern,2019-06-27,6.9,2019-01-02,0,28,33,0 days 04:00:00,237,german,urban,0
3,11729,11822,Indie - Outbound,Buffet,Bern,Bern,2019-06-27,6.9,2019-01-27,10,11,17,0 days 04:00:00,237,german,urban,0
4,11729,11822,Indie - Outbound,Buffet,Bern,Bern,2019-06-27,6.9,2019-01-31,10,13,17,0 days 04:00:00,237,german,urban,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2567223 entries, 0 to 2567222
Data columns (total 17 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Store ID                  int64  
 1   Item ID                   int64  
 2   Store Category            object 
 3   Store Segment             object 
 4   Store Region              object 
 5   City                      object 
 6   Item Last Supplying Date  object 
 7   Item Price                float64
 8   Date                      object 
 9   Declared Supply           int64  
 10  Meals Saved               int64  
 11  Total Supply              int64  
 12  Pickup Length             object 
 13  Supply Length             int64  
 14  Region Language           object 
 15  Region Type               object 
 16  Cluster                   int64  
dtypes: float64(1), int64(7), object(9)
memory usage: 333.0+ MB


In [7]:
# Converting column to datetime
df['Item Last Supplying Date'] =  pd.to_datetime(df['Item Last Supplying Date'], infer_datetime_format=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2567223 entries, 0 to 2567222
Data columns (total 17 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Store ID                  int64         
 1   Item ID                   int64         
 2   Store Category            object        
 3   Store Segment             object        
 4   Store Region              object        
 5   City                      object        
 6   Item Last Supplying Date  datetime64[ns]
 7   Item Price                float64       
 8   Date                      object        
 9   Declared Supply           int64         
 10  Meals Saved               int64         
 11  Total Supply              int64         
 12  Pickup Length             object        
 13  Supply Length             int64         
 14  Region Language           object        
 15  Region Type               object        
 16  Cluster                   int64         
dtypes: datet

In [9]:
# Computing the last day - 90 days a store supplied:
df['Item_trailing_3_m_supply_date']=df['Item Last Supplying Date']-pd.to_timedelta(90, unit='d')

In [10]:
# Check interval:
df[['Store ID','Item ID','Item Last Supplying Date','Item_trailing_3_m_supply_date']]

,Store ID,Item ID,Item Last Supplying Date,Item_trailing_3_m_supply_date
0,11344,11438,2022-03-01,2021-12-01
1,11729,11822,2019-06-27,2019-03-29
2,11729,11822,2019-06-27,2019-03-29
3,11729,11822,2019-06-27,2019-03-29
4,11729,11822,2019-06-27,2019-03-29
...,...,...,...,...
2567218,465374,446686,2022-02-26,2021-11-28
2567219,489610,466384,2022-02-28,2021-11-30
2567220,540352,521357,2021-12-02,2021-09-03
2567221,566347,546290,2022-01-04,2021-10-06


In [11]:
# Converting column to datetime
df['Date'] =  pd.to_datetime(df['Date'], infer_datetime_format=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2567223 entries, 0 to 2567222
Data columns (total 18 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   Store ID                       int64         
 1   Item ID                        int64         
 2   Store Category                 object        
 3   Store Segment                  object        
 4   Store Region                   object        
 5   City                           object        
 6   Item Last Supplying Date       datetime64[ns]
 7   Item Price                     float64       
 8   Date                           datetime64[ns]
 9   Declared Supply                int64         
 10  Meals Saved                    int64         
 11  Total Supply                   int64         
 12  Pickup Length                  object        
 13  Supply Length                  int64         
 14  Region Language                object        
 15  Region Type    

In [12]:
# For each item ID, dropping all the rows that have date prior to Item_trailing_3_m_supply_date:
df=df[df['Date']>=df['Item_trailing_3_m_supply_date']].reset_index()

In [13]:
# Check number of rows:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466461 entries, 0 to 466460
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   index                          466461 non-null  int64         
 1   Store ID                       466461 non-null  int64         
 2   Item ID                        466461 non-null  int64         
 3   Store Category                 466461 non-null  object        
 4   Store Segment                  466461 non-null  object        
 5   Store Region                   466461 non-null  object        
 6   City                           466461 non-null  object        
 7   Item Last Supplying Date       466461 non-null  datetime64[ns]
 8   Item Price                     466461 non-null  float64       
 9   Date                           466461 non-null  datetime64[ns]
 10  Declared Supply                466461 non-null  int64         
 11  

In [14]:
# Check:
df[['Store ID','Item ID','Date','Item Last Supplying Date','Item_trailing_3_m_supply_date']]

,Store ID,Item ID,Date,Item Last Supplying Date,Item_trailing_3_m_supply_date
0,11360,11454,2019-11-10,2020-02-01,2019-11-03
1,11360,11454,2019-11-11,2020-02-01,2019-11-03
2,11360,11454,2019-11-12,2020-02-01,2019-11-03
3,11360,11454,2019-11-13,2020-02-01,2019-11-03
4,11360,11454,2019-11-14,2020-02-01,2019-11-03
...,...,...,...,...,...
466456,465374,446686,2022-03-01,2022-02-26,2021-11-28
466457,489610,466384,2022-03-01,2022-02-28,2021-11-30
466458,540352,521357,2022-03-01,2021-12-02,2021-09-03
466459,566347,546290,2022-03-01,2022-01-04,2021-10-06


In [16]:
df[['Item ID','Item Last Supplying Date','Supply Length',
       'Region Language', 'Region Type', 'Cluster']][df['Item ID']==34].nunique()

Item ID                     1
Item Last Supplying Date    1
Supply Length               1
Region Language             1
Region Type                 1
Cluster                     1
dtype: int64

In [17]:
df.columns

Index(['index', 'Store ID', 'Item ID', 'Store Category', 'Store Segment',
       'Store Region', 'City', 'Item Last Supplying Date', 'Item Price',
       'Date', 'Declared Supply', 'Meals Saved', 'Total Supply',
       'Pickup Length', 'Supply Length', 'Region Language', 'Region Type',
       'Cluster', 'Item_trailing_3_m_supply_date'],
      dtype='object')

In [18]:
# Adding pick-up length:
# Fuction to convert:
def pickup_conv(x):
    x=str(x)
    x=x[len(x)-8:len(x)]
    h,m,s = x.split(':')
    x=int((datetime.timedelta(hours=int(h),minutes=int(m),seconds=int(s)).total_seconds())/60)
    return x

In [19]:
#Converting the whole column
df['Pickup Length']=df['Pickup Length'].map(pickup_conv)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466461 entries, 0 to 466460
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   index                          466461 non-null  int64         
 1   Store ID                       466461 non-null  int64         
 2   Item ID                        466461 non-null  int64         
 3   Store Category                 466461 non-null  object        
 4   Store Segment                  466461 non-null  object        
 5   Store Region                   466461 non-null  object        
 6   City                           466461 non-null  object        
 7   Item Last Supplying Date       466461 non-null  datetime64[ns]
 8   Item Price                     466461 non-null  float64       
 9   Date                           466461 non-null  datetime64[ns]
 10  Declared Supply                466461 non-null  int64         
 11  

In [23]:
# Aggregate by Item ID, Item_trailing_3_m_supply_date
df=df.groupby(['Store ID','Item ID','Store Category', 'Store Segment',
       'Store Region', 'City',
       'Item Last Supplying Date','Supply Length',
       'Region Language', 'Region Type', 'Cluster','Item_trailing_3_m_supply_date'])\
[['Item Price', 'Declared Supply', 'Meals Saved', 'Total Supply','Pickup Length']].mean().reset_index()
    #.sort_values('Population', ascending=False)


#df.groupby(['Metadata_A','treatment'],as_index=False).agg({'Metadata_B':'mean','ratio':'first'})
#df.groupby(['Metadata_A', 'treatment']).mean().reset_index()

In [24]:
# check:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9735 entries, 0 to 9734
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Store ID                       9735 non-null   int64         
 1   Item ID                        9735 non-null   int64         
 2   Store Category                 9735 non-null   object        
 3   Store Segment                  9735 non-null   object        
 4   Store Region                   9735 non-null   object        
 5   City                           9735 non-null   object        
 6   Item Last Supplying Date       9735 non-null   datetime64[ns]
 7   Supply Length                  9735 non-null   int64         
 8   Region Language                9735 non-null   object        
 9   Region Type                    9735 non-null   object        
 10  Cluster                        9735 non-null   int64         
 11  Item_trailing_3_m

In [25]:
# check:
df[['Item ID']].nunique()

Item ID    9735
dtype: int64

In [26]:
df[df.select_dtypes(include=[np.float]).columns.values.tolist()]=df.select_dtypes(include=[np.float]).round(2)

In [27]:
df

,Store ID,Item ID,Store Category,Store Segment,Store Region,City,Item Last Supplying Date,Supply Length,Region Language,Region Type,Cluster,Item_trailing_3_m_supply_date,Item Price,Declared Supply,Meals Saved,Total Supply,Pickup Length
0,35,34,Indie - Outbound,Bakery,Vaud,Salavaux,2022-03-01,999,french,urban,3,2021-12-01,6.9,3.14,3.05,3.29,20.00
1,57,73,Indie - Outbound,Bakery,Freiburg,Sâles,2022-03-01,997,french,urban,3,2021-12-01,6.9,1.94,2.09,2.26,30.00
2,58,74,Indie - Outbound,Bakery,Freiburg,Ursy,2022-02-26,976,french,urban,3,2021-11-28,4.9,0.00,1.33,1.37,30.00
3,58,337173,Indie - Outbound,Bakery,Freiburg,Ursy,2020-12-29,1,french,urban,3,2020-09-30,4.9,1.00,0.67,0.67,30.00
4,61,79,Indie - Outbound,Traditional Restaurant,Vaud,Lausanne,2022-02-26,960,french,urban,3,2021-11-28,6.9,1.00,0.97,1.00,30.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9730,658899,659462,Indie - Inbound,Fast Food / Take away restaurant,Genève,Genève,2022-03-01,7,french,urban,3,2021-12-01,6.9,1.00,4.71,4.71,81.43
9731,658899,662295,Indie - Inbound,Fast Food / Take away restaurant,Genève,Genève,2022-03-01,7,french,urban,3,2021-12-01,6.9,1.00,4.29,5.00,81.43
9732,659125,659681,Indie - Outbound,Gas station,Bern,Thun,2022-03-01,6,german,urban,0,2021-12-01,6.9,3.00,1.00,1.71,30.00
9733,662111,663086,Indie - Inbound,Fast Food / Take away restaurant,Zürich,Dietikon,2022-03-01,5,german,urban,0,2021-12-01,6.9,1.25,0.50,2.00,45.00


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9735 entries, 0 to 9734
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Store ID                       9735 non-null   int64         
 1   Item ID                        9735 non-null   int64         
 2   Store Category                 9735 non-null   object        
 3   Store Segment                  9735 non-null   object        
 4   Store Region                   9735 non-null   object        
 5   City                           9735 non-null   object        
 6   Item Last Supplying Date       9735 non-null   datetime64[ns]
 7   Supply Length                  9735 non-null   int64         
 8   Region Language                9735 non-null   object        
 9   Region Type                    9735 non-null   object        
 10  Cluster                        9735 non-null   int64         
 11  Item_trailing_3_m

In [29]:
df.to_csv('data/df_for_regression.csv')